<a href="https://colab.research.google.com/github/alenready/ML_AI_ICT-Assignments/blob/main/Assignments07_SentimentAnalysis_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Libraries

In [5]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer



In [6]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


In [7]:

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

In [3]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
df = pd.read_csv('/content/tweets.csv')
df

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [9]:
# prompt: make a copy of df

df_copy = df.copy()
df_copy


,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [10]:

# Basic information about the DataFrame
print(df.info())
print("\n---\n")
print(df.describe(include='all'))
print("\n---\n")
print("Shape of the DataFrame:", df.shape)
print("\n---\n")
print("First 5 rows:\n", df.head())
print("\n---\n")
print("Last 5 rows:\n", df.tail())
print("\n---\n")
print("Column names:", df.columns)
print("\n---\n")
print("Data types of each column:\n", df.dtypes)
print("\n---\n")
# Check for missing values
print("Missing Values:\n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7920 non-null   int64 
 1   label   7920 non-null   int64 
 2   tweet   7920 non-null   object
dtypes: int64(2), object(1)
memory usage: 185.8+ KB
None

---

                 id        label  \
count   7920.000000  7920.000000   
unique          NaN          NaN   
top             NaN          NaN   
freq            NaN          NaN   
mean    3960.500000     0.255808   
std     2286.451399     0.436342   
min        1.000000     0.000000   
25%     1980.750000     0.000000   
50%     3960.500000     0.000000   
75%     5940.250000     1.000000   
max     7920.000000     1.000000   

                                                    tweet  
count                                                7920  
unique                                               7918  
top     @architecture_3design - TAG YO

##Data Cleaning and Preprocessing

In [13]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

# Step 3: Data Cleaning and Preprocessing with Lemmatization
def clean_text(text):
    # Initialize WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Remove hashtags, mentions, URLs, special characters, and numbers
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'http\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Remove special characters and numbers
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing whitespace

    # Tokenize and lemmatize
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Apply text cleaning and lemmatization
df['cleaned_tweet'] = df['tweet'].apply(clean_text)
print(df.head(15))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


    id  label                                              tweet  \
0    1      0  #fingerprint #Pregnancy Test https://goo.gl/h1...   
1    2      0  Finally a transparant silicon case ^^ Thanks t...   
2    3      0  We love this! Would you go? #talk #makememorie...   
3    4      0  I'm wired I know I'm George I was made that wa...   
4    5      1  What amazing service! Apple won't even talk to...   
5    6      1  iPhone software update fucked up my phone big ...   
6    7      0  Happy for us .. #instapic #instadaily #us #son...   
7    8      0  New Type C charger cable #UK http://www.ebay.c...   
8    9      0  Bout to go shopping again listening to music #...   
9   10      0  Photo: #fun #selfie #pool #water #sony #camera...   
10  11      1  hey #apple when you make a new ipod dont make ...   
11  12      1  Ha! Not heavy machinery but it does what I nee...   
12  13      1  Contemplating giving in to the iPhone bandwago...   
13  14      0  I just made another crazy purchas

In [15]:
# Apply text cleaning
df['cleaned_tweet'] = df['tweet'].apply(clean_text)
print(df.head(25))

    id  label                                              tweet  \
0    1      0  #fingerprint #Pregnancy Test https://goo.gl/h1...   
1    2      0  Finally a transparant silicon case ^^ Thanks t...   
2    3      0  We love this! Would you go? #talk #makememorie...   
3    4      0  I'm wired I know I'm George I was made that wa...   
4    5      1  What amazing service! Apple won't even talk to...   
5    6      1  iPhone software update fucked up my phone big ...   
6    7      0  Happy for us .. #instapic #instadaily #us #son...   
7    8      0  New Type C charger cable #UK http://www.ebay.c...   
8    9      0  Bout to go shopping again listening to music #...   
9   10      0  Photo: #fun #selfie #pool #water #sony #camera...   
10  11      1  hey #apple when you make a new ipod dont make ...   
11  12      1  Ha! Not heavy machinery but it does what I nee...   
12  13      1  Contemplating giving in to the iPhone bandwago...   
13  14      0  I just made another crazy purchas

In [17]:

# cleaned tweets with full comments
for index, row in df.head().iterrows():
    print(f"Index: {index}")
    print(f"Original Tweet: {row['tweet']}")
    print(f"Cleaned Tweet: {row['cleaned_tweet']}")
    print("---")

Index: 0
Original Tweet: #fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
Cleaned Tweet: fingerprint pregnancy test  android apps beautiful cute health igers iphoneonly iphonesia iphone
---
Index: 1
Original Tweet: Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
Cleaned Tweet: finally a transparant silicon case  thanks to my uncle  yay sony xperia s sonyexperias
---
Index: 2
Original Tweet: We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
Cleaned Tweet: we love this would you go talk makememories unplug relax iphone smartphone wifi connect
---
Index: 3
Original Tweet: I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
Cleaned Tweet: im wired i know im george i was made that way  iphone cute

In [17]:

# Step 4: Splitting the Data into Train and Test Sets
X = df['cleaned_tweet']  # Features
y = df['label']  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train:\n", X_train)
print("\n---\n")
print("X_test:\n", X_test)
print("\n---\n")
print("y_train:\n", y_train)
print("\n---\n")
print("y_test:\n", y_test)

X_train:
 4252    cool car wash idea theisland bankholidaymonday...
4428    photo th birthday to the sony walkman at nobod...
7374    ipads are the biggest pile of fucking on the p...
1410    yearbook hmmmmm instagram instagood together f...
7896    so pissed macbook crash apple company doe noth...
                              ...                        
5226    shana tova jewish newyear everyone may your ne...
5390    im so sick of buying new cell phone charger ap...
860     it want it have it download the free iphone ap...
7603    photo nikosx iphone beach holiday bw iphone bl...
7270    just got an iphone s hehe iphone apple new fin...
Name: cleaned_tweet, Length: 6336, dtype: object

---

X_test:
 4896    photo cause we both dressed up today boyfriend...
7539    your product is brutal headphone always stop w...
1677    sunset today in zeeland samsung mobile s samsu...
1964    god it playstation share feature cutting my cl...
3025    awe he da bestest boyfriend him iloveyou cute ..

In [18]:
# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:

print("Shape of training data:", X_train.shape)
print("Shape of testing data:", X_test.shape)

Shape of training data: (6336,)
Shape of testing data: (1584,)


##Feature Extraction Using TF-IDF

In [21]:
# Step 5: Feature Extraction Using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [33]:
print(X_train_tfidf,'\n')
print('***************************')
print(X_test_tfidf)

  (0, 885)	0.15165902485863736
  (0, 629)	0.38988882910588474
  (0, 2044)	0.21145109577403604
  (0, 1211)	0.22075872712621797
  (0, 1000)	0.25974971496762445
  (0, 101)	0.25974971496762445
  (0, 200)	0.2095143193183189
  (0, 1522)	0.2511164854663033
  (0, 3743)	0.0849431916448262
  (0, 2214)	0.05429682565781049
  (0, 447)	0.25974971496762445
  (0, 463)	0.25974971496762445
  (0, 2479)	0.1999576440522258
  (0, 2467)	0.22361894934670704
  (0, 3682)	0.2135215455883944
  (0, 314)	0.2389486318936323
  (0, 4959)	0.2389486318936323
  (0, 2878)	0.25974971496762445
  (0, 4347)	0.16522292639480596
  (1, 3225)	0.23504005815788942
  (1, 4373)	0.3367340215735457
  (1, 418)	0.2669443246722878
  (1, 4460)	0.14530565012467878
  (1, 4388)	0.28217063141750304
  (1, 4048)	0.18135774428023324
  :	:
  (6333, 1564)	0.2607935640436819
  (6333, 1872)	0.2073658450211824
  (6333, 4216)	0.28175499811353144
  (6333, 4463)	0.21779523125970685
  (6333, 1148)	0.2934658563127054
  (6333, 4125)	0.44746811690641264
  (6

##Bag-Of-Words

In [38]:
# Bag-of-Words (Count Vectorization)
count_vectorizer = CountVectorizer(max_features=5000)
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

In [39]:

# Print the vocabulary (Bag-of-Words features)
print("Bag-of-Words Features:")
count_vectorizer.vocabulary_

Bag-of-Words Features:


{'cool': 885,
 'car': 629,
 'idea': 2044,
 'easter': 1211,
 'dave': 1000,
 'allen': 101,
 'april': 200,
 'fools': 1522,
 'samsung': 3743,
 'iphone': 2214,
 'bloggers': 447,
 'bmw': 463,
 'lg': 2479,
 'lenovo': 2467,
 'rock': 3682,
 'band': 314,
 'yoga': 4959,
 'muslim': 2878,
 'technology': 4347,
 'photo': 3225,
 'th': 4373,
 'birthday': 418,
 'to': 4460,
 'the': 4388,
 'sony': 4048,
 'walkman': 4749,
 'at': 247,
 'nobody': 2983,
 'beats': 352,
 'ipads': 2210,
 'are': 210,
 'biggest': 407,
 'pile': 3264,
 'of': 3035,
 'fucking': 1598,
 'on': 3053,
 'planet': 3289,
 'want': 4757,
 'throw': 4428,
 'thing': 4406,
 'wall': 4750,
 'instagram': 2159,
 'instagood': 2157,
 'together': 4466,
 'friendship': 1581,
 'smile': 3992,
 'so': 4013,
 'pissed': 3275,
 'macbook': 2617,
 'crashes': 925,
 'apple': 175,
 'company': 824,
 'does': 1108,
 'nothing': 3006,
 'applesucks': 188,
 'ripoff': 3675,
 'post': 3363,
 'good': 1720,
 'morning': 2826,
 'blog': 445,
 'blogger': 446,
 'cover': 912,
 'movile':

##Training Sentiment Classification Models

In [54]:
# prompt: list the words to analyse review

# Extract unique words from the cleaned tweets
all_words = []
for tweet in df['cleaned_tweet']:
    words = tweet.split()
    all_words.extend(words)

unique_words = list(set(all_words))

# Now unique_words holds a list of unique words from the cleaned tweets
print(unique_words[:100]) # Prints first 100 words as a sample

['mirager', 'pictwittercomkfusnmgp', 'knew', 'saas', 'coil', 'buddy', 'gtfcchink', 'hkig', 'igdaiy', 'trust', 'michaeljackson', 'chalk', 'sw', 'hatemyiphone', 'jehovahscreation', 'sarkozy', 'developing', 'creators', 'foto', 'trial', 'takemeout', 'served', 'domani', 'retrato', 'galeanastudio', 'quote', 'withyou', 'traitor', 'dapper', 'doesnt', 'h', 'sketch', 'kauai', 'samsun', 'despicable', 'bfs', 'alligatorskin', 'mmorpg', 'nailsbynettie', 'ripsocomconfrontation', 'instaxonlineph', 'chills', 'rosetta', 'messing', 'hoped', 'mug', 'ifitmodel', 'top', 'pools', 'sos', 'sadface', 'marilyn', 'aupair', 'donald', 'niceday', 'astounding', 'gluegelsolder', 'instapadinsta', 'sundaymorningpictwittercomxjbqkywbg', 'life', 'heathen', 'dd', 'nate', 'bandel', 'westchester', 'profit', 'giancarlo', 'giveawaypictwittercombronnplocp', 'dogsofaustin', 'godmorgen', 'idealofsweden', 'juda', 'watch', 'toogood', 'charlottesville', 'lovepictwittercomqekqdwgko', 'instatraveling', 'snapping', 'photoshoot', 'sonyn

In [22]:
# Step 6: Training Sentiment Classification Models
# Logistic Regression with TF-IDF
logistic_model = LogisticRegression()
logistic_model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [24]:
# Naive Bayes with Bag-of-Words
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train_bow, y_train)

# Random Forest with TF-IDF
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [23]:
# Step 7: Evaluating the Models
# Logistic Regression
y_pred_logistic = logistic_model.predict(X_test_tfidf)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy (TF-IDF): {logistic_accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred_logistic))


Logistic Regression Accuracy (TF-IDF): 0.88
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92      1152
           1       0.81      0.72      0.76       432

    accuracy                           0.88      1584
   macro avg       0.86      0.83      0.84      1584
weighted avg       0.88      0.88      0.88      1584



In [43]:
# Naive Bayes
y_pred_nb = naive_bayes_model.predict(X_test_bow)
nb_accuracy = accuracy_score(y_test, y_pred_nb)
print(f"Naive Bayes Accuracy (Bag-of-Words): {nb_accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred_nb))

Naive Bayes Accuracy (Bag-of-Words): 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.87      0.92      1152
           1       0.74      0.94      0.83       432

    accuracy                           0.89      1584
   macro avg       0.86      0.91      0.87      1584
weighted avg       0.91      0.89      0.90      1584



In [44]:
# Random Forest
y_pred_rf = random_forest_model.predict(X_test_tfidf)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy (TF-IDF): {rf_accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy (TF-IDF): 0.86
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.95      0.91      1152
           1       0.83      0.60      0.70       432

    accuracy                           0.86      1584
   macro avg       0.84      0.78      0.80      1584
weighted avg       0.85      0.86      0.85      1584



##Prediction on Sentiments

In [27]:

def predict_sentiment(new_tweets, model, vectorizer):
    cleaned_tweets = [clean_text(tweet) for tweet in new_tweets]
    transformed_tweets = vectorizer.transform(cleaned_tweets)
    predictions = model.predict(transformed_tweets)
    return predictions

# Example usage with Logistic Regression and TF-IDF
new_tweets = ["I love my new laptop!", "Terrible service at the store."]

# Define or retrieve the TfidfVectorizer object
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Or retrieve it if defined elsewhere
tfidf_vectorizer.fit(X_train) # Fit the vectorizer to your training data

predictions = predict_sentiment(new_tweets, logistic_model, tfidf_vectorizer)
print("Predicted Sentiments (Logistic Regression, TF-IDF):", predictions)

Predicted Sentiments (Logistic Regression, TF-IDF): [0 0]


In [28]:

# Example usage with Naive Bayes and Bag-of-Words
predictions_nb = predict_sentiment(new_tweets, naive_bayes_model, count_vectorizer)
print("Predicted Sentiments (Naive Bayes, Bag-of-Words):", predictions_nb)


Predicted Sentiments (Naive Bayes, Bag-of-Words): [0 1]


In [30]:
#  Prediction on new data

new_tweets = ["This is a great movie", "I hate this product"]
new_tweets_cleaned = [clean_text(tweet) for tweet in new_tweets]

# Initialize the TfidfVectorizer with the same parameters used during training
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Assuming you used max_features=5000 during training

# Assuming 'X' is your training data feature column used before
tfidf_vectorizer.fit(X) # If 'X' is not available, use the same data used during training


new_tweets_vectorized = tfidf_vectorizer.transform(new_tweets_cleaned)

new_predictions = logistic_model.predict(new_tweets_vectorized)  # Using logistic_model as an example
print(f"Predictions for new tweets: {new_predictions}")

Predictions for new tweets: [0 0]


#Prediction on multiple new rewiews, Checking whether Positive or Negative!

In [35]:
# Step 8: Predict Sentiment for New Reviews with Model Selection
def predict_multiple_reviews(new_reviews, model, vectorizer):
    cleaned_reviews = [clean_text(review) for review in new_reviews]
    transformed_reviews = vectorizer.transform(cleaned_reviews)
    predictions = model.predict(transformed_reviews)
    sentiment_map = {0: "Negative", 1: "Positive"}
    return [sentiment_map[pred] for pred in predictions]


In [36]:

# Predict  multiple new reviews
new_reviews = [
    "The product is fantastic and works perfectly!",
    "Worst purchase ever, it stopped working in a week.",
    "I am very happy with the service and quality.",
    "Terrible experience, will not buy again.",
    "Excellent! Highly recommended to everyone."
]

# Choose model and vectorizer (e.g., Logistic Regression and TF-IDF)
selected_model = logistic_model
selected_vectorizer = tfidf_vectorizer

# Predict sentiments for new reviews
predicted_sentiments = predict_multiple_reviews(new_reviews, selected_model, selected_vectorizer)

# Display the results
for review, sentiment in zip(new_reviews, predicted_sentiments):
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Review: The product is fantastic and works perfectly!
Predicted Sentiment: Negative

Review: Worst purchase ever, it stopped working in a week.
Predicted Sentiment: Negative

Review: I am very happy with the service and quality.
Predicted Sentiment: Negative

Review: Terrible experience, will not buy again.
Predicted Sentiment: Negative

Review: Excellent! Highly recommended to everyone.
Predicted Sentiment: Negative

